In [42]:
import wandb
import ultralytics
from ultralytics import YOLO
import torch
import cv2
import pandas as pd
import torch




In [43]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True" #to help with memory allocation issues(weak gpu)
torch.cuda.empty_cache() #weak gpu
#check yolo version and using cuda
# print(ultralytics.__version__, torch.cuda.is_available())

In [45]:
!nvidia-smi

Thu Feb  6 23:26:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:06:00.0  On |                  N/A |
| 32%   47C    P0             29W /  125W |    5637MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


|    0   N/A  N/A     16732    C+G   ...al\Discord\app-1.0.9181\Discord.exe      N/A      |
|    0   N/A  N/A     19340      C   ...johna\anaconda3\envs\fyp\python.exe      N/A      |
|    0   N/A  N/A     20084    C+G   ...64__8wekyb3d8bbwe\CalculatorApp.exe      N/A      |
|    0   N/A  N/A     20216    C+G   ...2.0_x64__cv1g1gvanyjgm\WhatsApp.exe      N/A      |
|    0   N/A  N/A     25848    C+G   ...Programs\Microsoft VS Code\Code.exe      N/A      |
+-----------------------------------------------------------------------------------------+


In [31]:
wandb.login()

True

Loading the model yoloV8 

In [32]:
#training model
model = YOLO('yolov8n.pt')


Arguments setting for the model

In [40]:
args = dict(
    data = "data.yaml",
    epochs = 10,
    patience = 5,
    batch = 4,
    save = True,
    save_period = 5,
    device = '0',
    workers = 4,
    project = "weights",
    name = "try",
    exist_ok = True, #overwrite the existing run for iterative testing, turn false when confident
    pretrained = True,
    optimizer = 'auto',
    resume = False,
    amp = False, #mixed precision training for faster training, might cause some issues
    fraction = 0.1, #fraction of dataset if resouce limited, change to 1 when confident
    profile = False, #enable ONNX and TensorRT profiling
    freeze = None, #freeze the first N layers or can specify the layers by index
    #for validation part
    # conf = 0.01,
    # iou = 0.6,
    # degrees = 0.0, 
    # shear = 0.0,
    # perspective = 0.0,
    plots = True,
    half = True, #use half precision for weaker gpu
)

In [41]:
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.3.72 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.9.21 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=5, batch=4, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=4, project=weights, name=try, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=0.1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 10.63 GiB is allocated by PyTorch, and 108.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)